In [1]:
%%writefile thrust_ex.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/iterator/counting_iterator.h>
#include <iostream>
#include <cuda_runtime.h>

// Function f(x) = 4.0 / (1 + x*x)
struct functor {
    __host__ __device__
    float operator()(const float& x) const {
        return 4.0 / (1.0 + x * x);
    }
};

// Function to be applied in transform_reduce
struct integral_functor {
    float dx;
    functor f;

    integral_functor(float _dx) : dx(_dx) {}

    __host__ __device__
    float operator()(const int& i) const {
        float x = (i + 0.5f) * dx;
        return f(x) * dx;
    }
};

int main() {
    const int N = 1000000000;
    const float dx = 1.0f / N;

    thrust::counting_iterator<int> begin(0);
    thrust::counting_iterator<int> end(N);

    float integral = thrust::transform_reduce(
        begin, end,
        integral_functor(dx),
        0.0f,
        thrust::plus<float>()
    );

    std::cout << "Approximate integral: " << integral << std::endl;

    return 0;
}


Writing thrust_ex.cu


In [2]:
!nvcc thrust_ex.cu -o thrust_ex
!./thrust_ex

Approximate integral: 3.14159
